In [0]:
from pyspark.sql.functions import sum as _sum, round, col

df = spark.table("sales_lakehouse.silver.product_sales_clean")

gold_df = (
    df.groupBy("order_date")
      .agg(
          _sum("revenue").alias("daily_revenue"),
          _sum("profit").alias("daily_profit"),
          _sum("quantity").alias("total_units_sold")
      )
      .withColumn(
          "profit_margin_pct",
          round((col("daily_profit") / col("daily_revenue")) * 100, 2)
      )
)

gold_df.write.mode("overwrite").format("delta") \
  .saveAsTable("sales_lakehouse.gold.daily_sales_kpi")

spark.table("sales_lakehouse.gold.daily_sales_kpi").display()
